In [64]:
# !pip3 install plotly numpy pandas retrying urllib3 matplotlib bokeh --user
import numpy as np
import pandas as pd
# import plotly
import json
token = "pk.eyJ1IjoiYXRoYXJ2YXMiLCJhIjoiY2tnZ3Q2dGV4MHpreDJzdDl2Ync4ODJvZyJ9.jFeZV_Mn0i5qufHmTmYYfQ"

from datetime import datetime
import re

# !pip install html5lib
!pip install emoji
import emoji
# !pip install bs4

import emoji
# !pip install bs4


     |████████████████████████████████| 51 kB 1.5 MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49714 sha256=98d8ba74052304003a9b726592062b43ad247ac2a909fb5ab20d3f59ad549db3
  Stored in directory: /home/atharvas/.cache/pip/wheels/43/3d/82/e7baffa5e86346c6178d7750dba6e8ef063282a37fc563f8f8
Successfully built emoji


In [1]:
procmoji = pd.read_csv("bigmoji.csv", lineterminator="\n")
with open("./torchMoji/data/emoji_codes.json", 'r') as s:
    num2emoji = json.load(s)


for col in ['Emoji_1','Emoji_2','Emoji_3','Emoji_4','Emoji_5']:
    procmoji[col] = procmoji[col].astype(int).astype(str).map(num2emoji).str.replace(":", "").apply(lambda x : emoji_to_sent[x]['Sentiment score[-1...+1]'])
procmoji['Sentiment'] = procmoji[['Emoji_1','Emoji_2','Emoji_3','Emoji_4','Emoji_5']].mean(axis=1)





NameError: name 'pd' is not defined

In [2]:
proc = pd.read_pickle("./big.pkl")
proc.head(2)

,full_text,created_at,id,long,lat,created_at_date,polarity
0,So there's going to be a hurricane tonight and...,Thu Oct 25 19:18:02 +0000 2012,261547183722082300,-80.062265,26.844194,2012-10-25 19:18:02+00:00,-0.2
1,They named the hurricane that will be coming u...,Thu Oct 25 19:18:42 +0000 2012,261547349921370100,-74.549170,40.618274,2012-10-25 19:18:42+00:00,-0.5


In [3]:
color_dict = {'TROPICAL DEPRESSION':'#AEF100', 'TROPICAL STORM':'#FFD600', 'HURRICANE-1':'#FF6440', 'HURRICANE-2':'#8506A9'}
import re
# Read data file, unzip from track.zip to get track.dat
fn  = 'track.dat'
rec = {'lat':[],'lon':[],'wind':[],'press':[],'dt':[],'cat':[]}
for i,line in enumerate(open(fn)):
    if i == 0: continue  # Jump over the first line
    # replace multiple whitespaces with a single whitespace
    line   = re.sub(r"\s+", ' ', line)
    pieces = line.split(" ")
    # retrieve information
    rec['lat'].append(float(pieces[0]))
    rec['lon'].append(float(pieces[1]))
    rec['wind'].append(float(pieces[3]))
    rec['press'].append(float(pieces[4]))
    rec['cat'].append((" ".join(pieces[5:])).strip())
    time   = pieces[2]
    time   = "2012/" + time
    rec['dt'].append(datetime.strptime(time,"%Y/%m/%d/%HZ"))
    
    
track = pd.DataFrame.from_dict(rec)

track.head(2)

,lat,lon,wind,press,dt,cat
0,13.5,-78.0,25.0,1003.0,2012-10-22 15:00:00,TROPICAL DEPRESSION
1,13.5,-78.5,25.0,1003.0,2012-10-22 18:00:00,TROPICAL DEPRESSION


In [4]:
times = pd.date_range(proc['created_at_date'].min().round("1H"), proc['created_at_date'].max().round("1H") + pd.DateOffset(hours=2), freq="2H")
times = [(times[i], times[i + 1]) for i in range(len(times) - 1)]
times[0][0].strftime("%Y-%m-%d %H:%M")
process_time = lambda tup : " (" + tup[0].strftime("%Y-%m-%d %H:%M") + ", " + tup[1].strftime("%Y-%m-%d %H:%M") + ")"


In [6]:
import plotly.graph_objects as go

mapbox_access_token = token

fig = go.Figure()

for start, end in times:
        selected_data = proc[(start < proc['created_at_date']) & (proc['created_at_date'] < end)]

        selected_data = selected_data[['lat', 'long', 'full_text', 'id', 'created_at', 'created_at_date', 'polarity']]

        selected_data['hover_text'] = selected_data['created_at'] + ': ' + selected_data['full_text'] 
        trace = go.Scattermapbox(
            lat=selected_data['lat'],
            lon=selected_data['long'],
            mode='markers',
            
            marker=go.scattermapbox.Marker(
                size=14, 
                color = selected_data['polarity'],
                colorscale= 'picnic',
                showscale=True,
                cmax=1.0,
                cmin=-1.0),
            text=selected_data['hover_text'],
            
#             name= start.strftime("%Y-%m-%d %H:%M")
        )
        
        fig.add_trace(trace)        
        
        

fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Tweets during time interval" + process_time(times[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Date: "},
    pad={"t": 50},
    steps=steps
)]



fig.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=45,
            lon=-73
        ),
        pitch=0,
        zoom=5
    ), 
    sliders=sliders

)


fig.write_html("./bigplot.html")

AttributeError: module 'retrying' has no attribute 'retry'